<a href="https://colab.research.google.com/github/hatemamine/Improving-Arabic-IR-and-Reranking-Performance-with-Knowledge-Distillation/blob/main/mrtydi_firststage_faiss_kd_vs_nokd_bi_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/facebookresearch/faiss/wiki/Running-on-GPUs

In [ ]:
!pip install faiss-cpu -q
!pip install huggingface_hub -q

In [ ]:
# embedding vector generated with our bi-encoder model
organization_dataset_id="hatemestinbejaia/ExperimentDATA_knowledge_distillation_vs_fine_tuning"
mrTydi_embedding_collection_list =["mrTydi-collection-arabic-embedding-KDaradpr", "mrTydi-collection-arabic-embedding-NOKDaradpr", "mrTydi-collection-arabic-embedding-mmarco-Arabic-AraElectra-bi-encoder-KD-v1", "mrTydi-collection-arabic-embedding-mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1", "mrTydi-collection-arabic-embedding-mmarco-Arabic-mMiniLML-bi-encoder-KD-v1", "mrTydi-collection-arabic-embedding-mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1"]
mrTydi_embedding_queries_list=["mrTydi-queries-arabic-embedding-KDaradpr", "mrTydi-queries-arabic-embedding-NOKDaradpr", "mrTydi-queries-arabic-embedding-mmarco-Arabic-AraElectra-bi-encoder-KD-v1", "mrTydi-queries-arabic-embedding-mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1", "mrTydi-queries-arabic-embedding-mmarco-Arabic-mMiniLML-bi-encoder-KD-v1", "mrTydi-queries-arabic-embedding-mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1"]

In [ ]:
import numpy as np
import gc
from datasets import load_dataset
output_run="first_stage_"
def combine_embedding(embedding1, embedding2):
    global output_run
    output_run+= mrTydi_embedding_collection_list[embedding1]+mrTydi_embedding_collection_list[embedding2]
    # load corpus embedding
    corpus_embeddings0 = load_dataset(organization_dataset_id, mrTydi_embedding_collection_list[embedding1])
    print(corpus_embeddings0)
    corpus_embeddings1 = load_dataset(organization_dataset_id, mrTydi_embedding_collection_list[embedding2])
    print(corpus_embeddings1)
    corpus_embeddings0=corpus_embeddings0["train"].with_format("np")
    corpus_id=corpus_embeddings0["docid"]
    corpus_embeddings0=corpus_embeddings0["embedding"]
    corpus_embeddings0
    # concatenate  embedding
    corpus_embeddings1=corpus_embeddings1["train"].with_format("np")
    corpus_embeddings1=corpus_embeddings1["embedding"]
    corpus_embeddings1
    corpus_embeddings =np.concatenate((corpus_embeddings0,corpus_embeddings1),1)
    del corpus_embeddings0
    del corpus_embeddings1
    # Force a garbage collection
    gc.collect()
    # load queries embedding
    queries_embeddings0 = load_dataset(organization_dataset_id, mrTydi_embedding_queries_list[embedding1], trust_remote_code=True)
    queries_embeddings1 = load_dataset(organization_dataset_id, mrTydi_embedding_queries_list[embedding2], trust_remote_code=True)
    queries_embeddings0 = queries_embeddings0["train"].with_format("np")
    queries_embeddings1 = queries_embeddings1["train"].with_format("np")
    queries_id=queries_embeddings0["id"]
    queries_embeddings0=queries_embeddings0["embedding"]
    queries_embeddings1=queries_embeddings1["embedding"]
    queries_embeddings =np.concatenate((queries_embeddings0,queries_embeddings1),1)
    #print(queries_embeddings)
    print(len(queries_embeddings[0]))
    return corpus_embeddings,corpus_id, queries_embeddings, queries_id

def load_embedding(embedding):
    global output_run
    output_run+= mrTydi_embedding_collection_list[embedding]
    # load corpus embedding
    corpus_embeddings = load_dataset(organization_dataset_id, mrTydi_embedding_collection_list[embedding])
    print(corpus_embeddings)
    corpus_embeddings=corpus_embeddings["train"].with_format("np")
    corpus_id=corpus_embeddings["docid"]
    corpus_embeddings=corpus_embeddings["embedding"]
    corpus_embeddings
    # Force a garbage collection
    gc.collect()
    # load queries embedding
    queries_embeddings = load_dataset(organization_dataset_id, mrTydi_embedding_queries_list[embedding], trust_remote_code=True)
    queries_embeddings = queries_embeddings["train"].with_format("np")
    queries_id=queries_embeddings["id"]
    queries_embeddings=queries_embeddings["embedding"]
    #print(queries_embeddings)
    print(len(queries_embeddings[0]))
    return corpus_embeddings,corpus_id, queries_embeddings, queries_id

In [ ]:
# you can combine embedding vector from diffrent bi-encoder model
combine=False
if combine:corpus_embeddings,corpus_id, queries_embeddings, queries_id=combine_embedding(0, 4)
else: corpus_embeddings,corpus_id, queries_embeddings, queries_id=load_embedding(4)

In [ ]:
"""
This example uses Approximate Nearest Neighbor Search (ANN) with FAISS (https://github.com/facebookresearch/faiss).
"""

import csv
import os
import pickle
import time

import faiss
import numpy as np

embedding_size = len(corpus_embeddings[0])  # Size of embeddings
# Defining our FAISS index
# Number of clusters used for faiss. Select a value 4*sqrt(N) to 16*sqrt(N) - https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index
#n_clusters = 10
# We use Inner Product (dot-product) as Index. We will normalize our vectors to unit length, then is Inner Product equal to cosine similarity
#quantizer = faiss.IndexFlatIP(embedding_size)
#print("quantizer")
#index = faiss.IndexIVFFlat(quantizer, embedding_size, n_clusters, faiss.METRIC_INNER_PRODUCT)
#print("index")
# Number of clusters to explorer at search time. We will search for nearest neighbors in 3 clusters.
#index.nprobe = 10

### Create the FAISS index
print("Start creating FAISS index")
#index = faiss.IndexIDMap(faiss.IndexFlatL2(embedding_size))
#index = faiss.IndexFlatL2(embedding_size)
index = faiss.index_factory(embedding_size, "Flat", faiss.METRIC_INNER_PRODUCT)
#corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1)[:, None]
faiss.normalize_L2(corpus_embeddings)
# First, we need to normalize vectors to unit length
#corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1)[:, None]
print("norm")
# Then we train the index to find a suitable clustering
index.train(corpus_embeddings)
print("train")

In [ ]:
faiss.write_index(index,"mrTydi.index")
index = faiss.read_index("mrTydi.index")
index.add(corpus_embeddings)
faiss.write_index(index, "mrTydi.index")
print("faiss index built it ok")
print("total index:",index.ntotal)

In [ ]:
#print(queries_embeddings[0])
top_k_hits = 1000
output_run += "_top_k_hits_"+str(top_k_hits) +".tsv"
print(output_run)

In [ ]:
corpus_id

In [ ]:
######### Search in the index ###########
import time
import numpy as np
from tqdm import trange
with open(output_run, 'w', encoding='utf-8') as f_out:
    for i in trange(len(queries_embeddings)):
        start_time = time.time()
        question_embedding = queries_embeddings[i]
        # FAISS works with inner product (dot product). When we normalize vectors to unit length, inner product is equal to cosine similarity
        #question_embedding = question_embedding / np.linalg.norm(question_embedding)
        question_embedding = np.expand_dims(question_embedding, axis=0)
        faiss.normalize_L2(question_embedding)

        # Search in FAISS. It returns a matrix with distances and corpus ids.
        distances, corpus_ids = index.search(question_embedding, top_k_hits)

        # We extract corpus ids and scores for the first query
        hits = [{"corpus_id": id, "score": score} for id, score in zip(corpus_ids[0], distances[0])]
        #print(hits)
        hits = sorted(hits, key=lambda x: x["score"], reverse=True)
        end_time = time.time()
        for j in range(len(hits)):
            f_out.write(f'{queries_id[i]}\t{corpus_id[hits[j]["corpus_id"]]}\t{j+1}\n')
        #break

In [ ]:
print(end_time-start_time)

In [ ]:
!pip install -U datasets huggingface-hub -q
import huggingface_hub
hf = huggingface_hub.HfFolder()
# to load the reranked list to your Hugging Face repository, enter your access token below.
# and change the repository ID to yours.
#organization_dataset_id =""
access_token = "Hagging face access token"
hf.save_token(access_token)
from huggingface_hub import HfApi
api = HfApi()
#api.upload_file(path_or_fileobj=output_run, path_in_repo=output_run, repo_id=organization_dataset_id, repo_type="dataset",)